<div style="  padding: 10px;text-align: center;" class='row'>
<div style="float:left;width: 10%;" class='column'><a href="https://datos.gov.co/"><img alt="Logo DataSandbox"  src="https://github.com/DataSandbox/Plantilla-Publicacion-Resultados/raw/main/App/logdat.JPG" style="width: 100px;"></a></div>
    <div style="float:left;width: 80%;" class='column'>
        <h1>EXTRACCIÓN DE INFORMACIÓN EN DOCUMENTOS DE PROCESOS DE CONTRATACIÓN PÚBLICA
        </h1> 
    </div>
 <div style="float:left;width: 10%;" class='column'><a href="https://www.colombiacompra.gov.co/" target="_blank"><img class="float-right" src="https://raw.githubusercontent.com/ANCP-CCE-Analitica/datasandbox-extraccion/main/logo_ancp_cce_web.png" style="width: 200px;"></a></div>
    </div>

# Obtención de muestra de contratos del secop

In [ ]:
import pandas as pd
from sodapy import Socrata
pd.set_option("display.max_columns", None)

## EDA datos SECOP I
### Carga de datos completos, SECOP hasta el 06/10/2021

Para obtener los contratos del SECOP I, se toma la base de datos de SECOP I en su totalidad y de esta se saca una muestra con base en la modalidad de contratación y el nombre de la entidad. La siguiente pieza de código muestra como se puede hacer a traves de `socrata`para mayor información consultar en [https://dev.socrata.com/foundry/www.datos.gov.co/xvdy-vvsk](https://dev.socrata.com/foundry/www.datos.gov.co/xvdy-vvsk)

In [ ]:
client = Socrata("www.datos.gov.co",
                 "App_Token",
                 username="example@mail.com",
                 password="PassWord")
results = client.get("xvdy-vvsk", limit=1300000)
df = pd.DataFrame.from_records(results)

### Visualización de las columnas y conteo de frecuencias

Para entender el comportamiento y la distribución de los datos, miramos la cantidad de contratos y procesos en los que participaron las diferentes entidades y cómo se ditribuyen estos por la modalidad de contratación. De esta forma, vemos que existen un grán número de entidades que han participado en múltiples procesos, sin embargo así mismo existen varias entidades que han participado en pocos procesos o incluso 1. Similar a esto, se puede observar que la mayoría de los procesos se encuentran en 11 de las diferentes modalidades, los cuales tienen más de 10 mil procesos cada una.

Usamos la base `Mapeo_Variables_SECOPI.xlsx ` para usar los nombres de las variables registrados en el diccionario de datos.

In [ ]:
DF=pd.read_excel('Mapeo_Variables_SECOPI.xlsx')
new_columns=[]
for i in df.columns:
    h=0
    for j in DF['Socrata'].index:
        if i==DF['Socrata'][j]:
            h=1
            new_columns.append(DF['Datos Abiertos'][j])
    if h==0:
        print(i)
        new_columns.append(i)
        
df.columns=new_columns

Las variables que no se actualizan según el diccionario de datos apareceran impresas.

Veamos la distribución según las entidades:

In [ ]:
df["Nombre de la Entidad"].value_counts()

Según el tipo de proceso:

In [ ]:
df["Tipo de Proceso"].value_counts()

### Borrado de datos repetidos por "Nombre de entidad" y "Tipo de Proceso"

Para obtener una mayor variedad en el tipo de contrato, en su formato y en el archivo del contrato que contiene el proceso, se reducen los datos de tal forma que solo se guarde un contrato para cada entidad por cada modalidad de contratación. Es decir que una misma entidad puede tener varios contratos, pero solo uno de un mismo tipo de modalidad de contratación.

In [ ]:
df = df.drop_duplicates(subset=["Nombre de la Entidad", "Tipo de Proceso"]) #11

Luego de reducir los contratos repetidos por entidad y modalidad de contratación, se observa la distribución de los contratos por la modalidad de contratación para obtener los procesos que son más recurrentes y que, de esta forma se pueda sacar la muestra de los contratos más representativos.

In [ ]:
df["Tipo de Proceso"].value_counts()

## Partición en grupos por Modalidad de contratación
Se toman los contratos que tienen más de 10,000 contratos en su modalidad antes de ser filtrados, con el fin de usar los tipos de contratos que son más relevantes y frecuentes, sin dividir en exceso los datos en grupos con muy pocos contratos.<br>
De estos grupos de saca una muestra aleatoria de 2000 contratos en los grupos que se puede y se toma el grupo completo en los que tienen menos de 2000 contratos.<br>
Por los tanto los contratos de los que se toma la muestra son los siguientes:<br>
<ul>
    <li>Contratación Directa (Ley 1150 de 2007)</li>
    <li>Régimen Especial</li>
    <li>Contratación Mínima Cuantía</li>
    <li>Selección Abreviada de Menor Cuantía (Ley 1150 de 2007)</li>
    <li>Subasta</li>
    <li>Licitación Pública</li>
    <li>Concurso de Méritos Abierto</li>
    <li>Contratación Directa Menor Cuantía</li>
    <li>Otras Formas de Contratación Directa</li>
    <li>Contratos y convenios con más de dos partes</li>
    <li>Licitación obra pública</li>
</ul>

## Muestreo aleatorio de 2000 contratos por "Tipo de Proceso"
Se extrae una muestra de 2000 contratos aleatorios de cada "tipo de Proceso". Sin embargo, se observa que luego de borrar los duplicados el número de contratos en varias modalidades de contratación disminuye por debajo de la cantidad de la muestra, por lo tanto se toman estos contratos en su totalidad.

In [ ]:
Procesos_2000_mas=["Contratación Directa (Ley 1150 de 2007)", "Régimen Especial",
                   "Contratación Mínima Cuantía","Selección Abreviada de Menor Cuantía (Ley 1150 de 2007)",
                   "Subasta", "Licitación Pública","Concurso de Méritos Abierto","Contratación Directa Menor Cuantía",
                   "Otras Formas de Contratación Directa","Contratos y convenios con más de dos partes","Licitación obra pública"]
Lista_tipo_de_procesos=[]
for i in Procesos_2000_mas:
    data=df[df["Tipo de Proceso"] == i]
    Lista_tipo_de_procesos.append(data.sample(2000))                           

## Unión de los dataframes y exportar los datos
Para finalizar, las diferentes muestras son agrupadas en un mismo archivo, obteniendo 22,021 procesos a los cuales se le va a extraer el contrato, en caso de tener registrado en ese proceso un contrato.

In [ ]:
df_final = pd.concat(Lista_tipo_de_procesos, ignore_index = True)

In [ ]:
df_final.head()

In [ ]:
df_final.shape

In [ ]:
df_final["Codigo_contrato"]= df_final["Numero de Constancia"]

In [ ]:
df_final.to_csv("muestra_SECOP_I.csv", index = False)

In [ ]:
lista_cod_contrato = df_final[["Codigo_contrato"]]

Un trabajo con esta lista final nos permite utilizar un API de la entidad para la descarga masiva de contratos en PDF, en la carpeta contratos encontrarán algunos ejemplos, para la implementación que se hizo en este ejercicio se trabajaron con los contratos que aparecen en la base `contratos_procesados.csv`.

In [ ]:
contratos_sql = pd.concat([contratos_sql1, contratos_sql2, contratos_sql3, contratos_sql4, contratos_sql5, contratos_sql6, contratos_sql7, contratos_sql8, contratos_sql9, contratos_sql10, contratos_sql11, contratos_sql12, contratos_sql13, contratos_sql14, contratos_sql15, contratos_sql16, contratos_sql17, contratos_sql18, contratos_sql19, contratos_sql20, contratos_sql21, contratos_sql22])

De esta forma observamos que luego de extraer los contratos de la muestra anterior, el total de la muestra disminuye y obtenemos un total de la muestra de 14,693 contratos extraidos.